# Similarity Measures
### Functions measuring similarity using graph edit distance.
#### The graph edit distance is the number of edge/node changes needed to make two graphs isomorphic.

In [1]:
import os, glob
import time, datetime
import csv
import networkx as nx
import simplejson as json
from pathlib import Path

def save_graph(graph, fname):
    nodes = [{'id': n, 'group': graph.nodes[n]['group'], 'degree': str(graph.nodes[n]['degree'])} for n in graph.nodes()]
    links = [{'source': u, 'target': v, 'label': d['label']} for u, v, d in graph.edges(data=True)]
    with open(fname, 'w') as f:
        json.dump({'nodes': nodes, 'links': links}, f, indent=4,)
    #print('Graph file %s is created.', fname)

def load_graph(filename, name):
    d = json.load(open(filename))
    g = nx.DiGraph(name = name)
    for n in d['nodes']:
        if n['group'] != 'OTHER':
            g.add_node(n['id'], group = n['group'], degree = n['degree'])
    for n in d['links']:
        g.add_edge(n['source'], n['target'], label = n['label'])
    return g

start_all_time = time.time()
print('STARTED: ', datetime.datetime.now())

STARTED:  2020-07-01 10:29:59.111143


#### Graph edit distance is a graph similarity measure analogous to Levenshtein distance for strings.
It is defined as minimum cost of edit path (sequence of node and edge edit operations)
transforming graph G1 to graph isomorphic to G2.

graph_edit_distance(G1, G2, node_match=None, edge_match=None, node_subst_cost=None, node_del_cost=None,
                    node_ins_cost=None, edge_subst_cost=None, edge_del_cost=None, edge_ins_cost=None, upper_bound=None)

In [4]:
def calculate_GAD(folder_name):
    files = glob.glob(os.path.join(folder_name, '*.json'))
    print("Total number of input files # = ", len(files))
    graph_edit_distances = []
    
    for i in range(0, len(files)-1):
        G1 = load_graph(files[i], 'G1')
        print(f'\n=================================\nFILE 1: {Path(files[i]).name} - Graph ', nx.info(G1))

        for j in range(1, len(files)):
            try:
                start_time = time.time()

                G2 = load_graph(files[j], 'G2')
                print(f'\nFILE 2: {Path(files[j]).name} - Graph', nx.info(G2))

                graph_edit_distance = nx.optimize_graph_edit_distance(G1, G2)
                
                for v in graph_edit_distance:
                    graph_edit_distances.append([v, Path(files[i]).name, Path(files[j]).name])
                    print(f"\nGraph edit distance is {v}")
                    break

                print("Duration = ", time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))

            except Exception as ex:
                print("Error msg: " + str(ex))
                count_files_error += 1

    hours, rem = divmod(time.time() - start_all_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Total processing time: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    print('FINISHED: ', datetime.datetime.now())
    return graph_edit_distances

In [5]:
# Gold graphs
graph_edit_distances_gold = calculate_GAD(r"../wamex_data/wamex_graphs_fastest/gold/")
print(graph_edit_distances_gold)

Total number of input files # =  4

FILE 1: gold_a084492_coolgardie_annual_report_2009_15818118.json - Graph  Name: G1
Type: DiGraph
Number of nodes: 28
Number of edges: 36
Average in degree:   1.2857
Average out degree:   1.2857

FILE 2: gold_a074513_coolgardie_combined_annual_2006_final_13997651.json - Graph Name: G2
Type: DiGraph
Number of nodes: 178
Number of edges: 678
Average in degree:   3.8090
Average out degree:   3.8090

Graph edit distance is 842.0
Duration =  00:33:31

FILE 2: gold_a072821_coolgardie_combined_annual_2005_12942716.json - Graph Name: G2
Type: DiGraph
Number of nodes: 152
Number of edges: 551
Average in degree:   3.6250
Average out degree:   3.6250

Graph edit distance is 697.0
Duration =  00:13:05

FILE 2: gold_a076168_coolgardie annual report 2007_9968549.json - Graph Name: G2
Type: DiGraph
Number of nodes: 29
Number of edges: 40
Average in degree:   1.3793
Average out degree:   1.3793

Graph edit distance is 69.0
Duration =  00:00:00

FILE 1: gold_a074513_c

In [16]:
print('Graph Edit Distance Scores:')
for score, f1, f2 in sorted(graph_edit_distances_gold):
    if score:
        print(score, '\n', f1, f2)

Graph Edit Distance Scores:
69.0 
 gold_a084492_coolgardie_annual_report_2009_15818118.json gold_a076168_coolgardie annual report 2007_9968549.json
697.0 
 gold_a084492_coolgardie_annual_report_2009_15818118.json gold_a072821_coolgardie_combined_annual_2005_12942716.json
698.0 
 gold_a072821_coolgardie_combined_annual_2005_12942716.json gold_a076168_coolgardie annual report 2007_9968549.json
842.0 
 gold_a084492_coolgardie_annual_report_2009_15818118.json gold_a074513_coolgardie_combined_annual_2006_final_13997651.json
857.0 
 gold_a074513_coolgardie_combined_annual_2006_final_13997651.json gold_a076168_coolgardie annual report 2007_9968549.json
1137.0 
 gold_a072821_coolgardie_combined_annual_2005_12942716.json gold_a074513_coolgardie_combined_annual_2006_final_13997651.json
1157.0 
 gold_a074513_coolgardie_combined_annual_2006_final_13997651.json gold_a072821_coolgardie_combined_annual_2005_12942716.json


In [17]:
# Iron ore graphs
graph_edit_distances_iron_ore = calculate_GAD(r"../wamex_data/wamex_graphs_fastest/iron_ore/")
print(graph_edit_distances_iron_ore)

Total number of input files # =  7

FILE 1: a078606_c125-1997-2008a_12703370.json - Graph  Name: G1
Type: DiGraph
Number of nodes: 90
Number of edges: 169
Average in degree:   1.8778
Average out degree:   1.8778

FILE 2: a072391_c125_2004_2006a_12728776.json - Graph Name: G2
Type: DiGraph
Number of nodes: 203
Number of edges: 730
Average in degree:   3.5961
Average out degree:   3.5961


KeyboardInterrupt: 

In [ ]:
print(*sorted(graph_edit_distances_iron_ore), sep='\n')